***This code is adapted from: https://github.com/victoria-ianeva/NERA-Tutorial-on-linguistic-feature-extraction***

It extracts linguistic features from the datasets and then uses them to predict the target value (difficulty, facility or response time) for our 3 datasets(bea, bio, cmcqrd)
***Readability metrics are not applied, as most question items have fewer than 100 words (https://github.com/cdimascio/py-readability-metrics)***

In [6]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from collections import Counter
import re
import spacy
import en_core_web_sm
import glob
from spacy.matcher import Matcher
from spacy.util import filter_spans

nlp = en_core_web_sm.load()

# Functions

In [7]:
def preprocess(text):
    '''This is a function to perform tokenization, lemmatization, removal of non-alphabetic characters
    and stopword removal'''
    stopwords = ['a', 'an', 'the', 'with', 'to', 'be', 'have']
  	# Create Doc object
    doc = nlp(text, disable=['ner'])
    # Generate lemmas
    lemmas = [token.lemma_ for token in doc]
    # Remove stopwords and non-alphabetic characters
    a_lemmas = [lemma for lemma in lemmas 
            if lemma.isalpha() and lemma not in stopwords]
    return ' '.join(a_lemmas)

def count_words(string):
    '''This function returns the number of words in a string'''
    # Split the string into words
    words = string.split()
    # Return the number of words
    return len(words)
  
def word_length(string):
    '''This function returns the average word length in characters for the words in an item'''
    #Get the length of the full text in characters
    chars = len(string)
    #Split the string into words
    words = string.split()
    #Compute the average word length and round the output to the second decimal point
    avg_word_length = chars/len(words)
    return round(avg_word_length, 2)
  
def sentence_counter(text):
    '''This function returns the number of sentences in an item'''
    doc = nlp(text)
    #Initialize a counter variable
    counter = 0
    #Update the counter for each sentence which can be found in the doc.sents object returned by the Spacy model
    for sentence in doc.sents:
        counter = counter + 1
    return counter

def avg_sent_length(text):
    '''This function returns the average sentence length in an item'''
    doc = nlp(text)
    #Initialize a counter variable
    sent_number = 0
    #Update the counter for each sentence which can be found in the doc.sents object returned by the Spacy model
    for sent in doc.sents:
        sent_number = sent_number + 1
    #Get the number of words
    words = text.split()
    #Compute the average sentence length and round it to the second decimal point
    avg_sent_length = len(words)/sent_number
    return round(avg_sent_length, 2)

def nouns(text, model=nlp):
    '''This function returns the number of nouns in an item'''
    # Create doc object 
    doc = model(text)
    # Generate list of POS tags
    pos = [token.pos_ for token in doc]
    # Return number of nouns
    return pos.count('NOUN')

def verbs(text, model=nlp):
    '''This function returns the number of verbs in an item'''
    # Create doc object
    doc = model(text)
    # Generate list of POS tags
    pos = [token.pos_ for token in doc]
    # Return number of verbs
    return pos.count('VERB')

def adjectives(text, model=nlp):
    '''This function returns the number of adjectives in an item'''
    # Create doc object
    doc = model(text)
    # Generate list of POS tags
    pos = [token.pos_ for token in doc]
    # Return number of adjectives
    return pos.count('ADJ')

def adverbs(text, model=nlp):
    '''This function returns the number of adverbs in an item'''
    # Create doc object
    doc = model(text)
    # Generate list of POS tags
    pos = [token.pos_ for token in doc]
    # Return number of adverbs
    return pos.count('ADV')

def get_nps(text):
    '''This is a function that outputs the number of noun phrases in an item'''
    doc = nlp(text)
    NP_count = 0
    for np in doc.noun_chunks:
        NP_count = NP_count + 1
    return NP_count

def get_pps(text):
    '''This is a function that outputs the number of prepositional phrases in an item'''
    doc = nlp(text)
    pps = 0
    for token in doc:
        # You can try this with other parts of speech for different subtrees.
        if token.pos_ == 'ADP':
            
            #Use the command below if you wanted to get the actual PPs
            #pp = ' '.join([tok.orth_ for tok in token.subtree])
            
            #This command counts the number of PPs
            pps = pps + 1
    return pps

def get_vps(text):
    '''This function returns the number of verb phrases in an item'''
    #We can modify this pattern for the extraction of verb phrases
    pattern = [{'POS': 'VERB', 'OP': '?'},
            {'POS': 'ADV', 'OP': '*'},
            {'POS': 'AUX', 'OP': '*'},
            {'POS': 'VERB', 'OP': '+'}]
    doc = nlp(text)
    vps = 0
    # instantiate a Matcher instance
    matcher = Matcher(nlp.vocab)
    matcher.add("Verb phrase", [pattern], on_match=None)
    # call the matcher to find matches 
    matches = matcher(doc)
    spans = [doc[start:end] for _, start, end in matches]
    for match in matches:
        vps = vps +1
    return vps

#Connectives to instruct, recount and sequence
temporal_connectives = ['afterwards', 'once', 'at this moment', 'at this point', 'before', 'finally', 
                        'here', 'in the end', 'lastly', 'later on', 'meanwhile', 'next', 'now', 
                        'on another occasion', 'previously','since', 'soon', 'straightaway', 'then', 
                        'when', 'whenever', 'while']


#Connectives to show cause or conditions
causal_connectives = ['accordingly', 'all the same', 'an effect of', 'an outcome of', 'an upshot of',
                      'as a consequence of', 'as a result of', 'because', 'caused by', 'consequently',
                      'despite this', 'even though', 'hence', 'however', 'in that case', 'moreover',
                      'nevertheless', 'otherwise', 'so', 'so as', 'stemmed from', 'still', 'then',
                      'therefore', 'though', 'under the circumstances', 'yet']


#Connectives for showing results
exemplifying_connectives = ['accordingly', 'as a result', 'as exemplified by', 'consequently', 'for example',
                            'for instance', 'for one thing', 'including', 'provided that', 'since', 'so',
                            'such as', 'then', 'therefore', 'these include', 'through', 'unless', 'without']


#Connectives to show similarity or add a point
additive_connectives = ['and', 'additionally', 'also', 'as well', 'even', 'furthermore', 'in addition', 'indeed',
                        'let alone', 'moreover', 'not only']

#Connectives showing a difference or an opposite point of view
contrastive_connectives = ['alternatively', 'anyway', 'but', 'by contrast', 'differs from', 'elsewhere',
                           'even so', 'however', 'in contrast', 'in fact', 'in other respects', 'in spite of this',
                           'in that respect', 'instead', 'nevertheless', 'on the contrary', 'on the other hand',
                           'rather', 'though', 'whereas', 'yet']
def temporal_connectives_count(text):
    '''This function counts the number of temporal connectives in a text'''
    count = 0
    for string in temporal_connectives:
        for match in re.finditer(string, text):
            count +=  1
    return count
def causal_connectives_count(text):
    '''This function counts the number of causal connectives in a text'''
    count = 0
    for string in causal_connectives:
        for match in re.finditer(string, text):
            count +=  1
    return count

def exemplifying_connectives_count(text):
    '''This function counts the number of exemplifying connectives in a text'''
    count = 0
    for string in exemplifying_connectives:
        for match in re.finditer(string, text):
            count +=  1
    return count

def additive_connectives_count(text):
    '''This function counts the number of additive connectives in a text'''
    count = 0
    for string in additive_connectives:
        for match in re.finditer(string, text):
            count +=  1
    return count

def contrastive_connectives_count(text):
    '''This function counts the number of contrastive connectives in a text'''
    cont_con = 0
    for string in contrastive_connectives:
        if string in text:
            cont_con = cont_con + 1
    return cont_con

In [8]:
def apply_all_functions(df):
    print("Step 1")
    df['question_with_options'] = df['question_with_options'].str.lower()
    df['Item'] = df['question_with_options'].apply(preprocess)
    df['Word_Count'] = df['question_with_options'].apply(count_words)
    df['Word_Count_No_stop_words'] = df['Item'].apply(count_words)
    df['Avg_Word_Length'] = df['Item'].apply(word_length)
    df['Sentence_Count'] = df['question_with_options'].apply(sentence_counter)
    df['Avg_Sent_Length_in_Words'] = df['question_with_options'].apply(avg_sent_length)
    print("Step 2")
    df['Noun_Count'] = df['Item'].apply(nouns)
    df['Verb_Count'] = df['Item'].apply(verbs)
    df['Adjective_Count'] = df['Item'].apply(adjectives)
    df['Adverb_Count'] = df['Item'].apply(adverbs)
    df['Number_of_NPs'] = df['Item'].apply(get_nps)
    df['Number_of_PPs'] = df['Item'].apply(get_pps)
    df['Number_of_VPs'] = df['Item'].apply(get_vps)
    print("Step 3")
    df['Temporal_Connectives_Count'] = df['question_with_options'].apply(temporal_connectives_count)
    df['Causal_Connectives_Count'] = df['question_with_options'].apply(causal_connectives_count)
    df['Exemplifying_Connectives_Count'] = df['question_with_options'].apply(exemplifying_connectives_count)
    df['Additive_Connectives_Count'] = df['question_with_options'].apply(additive_connectives_count)
    df['Contrastive_Connectives_Count'] = df['question_with_options'].apply(contrastive_connectives_count)
    print("Step 4")
    
    return df

# Apply for all datasets and splits

In [9]:
DATASETS = ['bio','cmcqrd','usmle']
SPLITS = ['train', 'test']

In [ ]:
for dataset in DATASETS:
    for split in SPLITS:
        print("Dataset: ", dataset, "Split: ", split)
        df = pd.read_csv(f'../data/{dataset}/preprocessed/combined_results_{split}_set.csv')
        df = apply_all_functions(df)
        df.to_csv(f'../data/{dataset}/with_ling_features/{split}.csv', index=False)
        
print("Done!")